## Food Cutting OWLReady Integration Notebook

OWLReady Integration with owlready2 for the Food Cutting SPARQL Queries from: https://github.com/Food-Ninja/CuttingFood/blob/main/jupyter/FoodCuttingQueries.ipynb
- The URL can be a file from the internet but also a local file
- Defining the Namespaces globally so they can be accessed by every function
- Accessing a class requires to call the Namespace for example, the class "apple" (IRI: "http://purl.obolibrary.org/obo/FOODON_03301710") corresponds to OBO.FOODON_03301710, as the OBO namespace is defined as such.

In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox
import owlready2
from owlready2 import get_ontology


url = "https://raw.githubusercontent.com/Food-Ninja/CuttingFood/main/owl/food_cutting.owl"
onto = get_ontology(url).load()
SOMA = onto.get_namespace("http://www.ease-crc.org/ont/SOMA.owl#")
CUT2 = onto.get_namespace("http://www.ease-crc.org/ont/situation_awareness#")
CUT = onto.get_namespace("http://www.ease-crc.org/ont/food_cutting#")
DUL = onto.get_namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
OBO = onto.get_namespace("http://purl.obolibrary.org/obo/")

In [ ]:
# all available parameters
tasks = [('Cutting Action',"http://www.ease-crc.org/ont/food_cutting#CuttingAction    "),
        ('Quartering', "http://www.ease-crc.org/ont/food_cutting#Quartering"),
        ('Julienning',"http://www.ease-crc.org/ont/food_cutting#Julienning"),
        ('Halving',"http://www.ease-crc.org/ont/food_cutting#Halving"),
        ('Dicing',"http://www.ease-crc.org/ont/SOMA.owl#Dicing"),
        ('Cutting',"http://www.ease-crc.org/ont/SOMA.owl#Cutting"),
        ('Slicing',"http://www.ease-crc.org/ont/SOMA.owl#Slicing"),
        ('Snipping',"http://www.ease-crc.org/ont/food_cutting#Snipping"),
        ('Slivering',"http://www.ease-crc.org/ont/food_cutting#Slivering"),
        ('Sawing',"http://www.ease-crc.org/ont/food_cutting#Sawing"),
        ('Paring',"http://www.ease-crc.org/ont/food_cutting#Paring"),
        ('Carving',"http://www.ease-crc.org/ont/food_cutting#Carving"),
        ('Mincing',"http://www.ease-crc.org/ont/food_cutting#Mincing"),
        ('Cubing',"http://www.ease-crc.org/ont/food_cutting#Cubing"),
        ('Chopping',"http://www.ease-crc.org/ont/food_cutting#Chopping")]

objects=[('almond', "obo:FOODON_00003523"),
        ('apple', "obo:FOODON_03301710"),
        ('avocado', "obo:FOODON_00003600"),
        ('banana', "obo:FOODON_00004183"),
        ('bean', "obo:FOODON_03301403"),      
        ('cherry', "obo:FOODON_03301240"),
        ('citron', "obo:FOODON_03306596"),
        ('coconut', "obo:FOODON_00003449"),     
        ('cucumber', "obo:FOODON_00003415"),
        ('kiwi', "obo:FOODON_00004387"), 
        ('kumquat', "obo:FOODON_03306597"),
        ('lemon', "obo:FOODON_03301441"),
        ('lime', "obo:FOODON_00003661"),
        ('olive', "obo:FOODON_03317509"),
        ('orange', "obo:FOODON_03309832"),
        ('peach', "obo:FOODON_03315502"), 
        ('pepper', "obo:FOODON_00003520"),
        ('pineapple', "obo:FOODON_00003459"),
        ('pumpkin', "obo:FOODON_00003486"),
        ('strawberry', "obo:FOODON_00003443"),        
        ('squash', "obo:FOODON_00003539"),
        ('tomato', "obo:FOODON_03309927")]

task=""
tobject=""

# Takes a Task of the tasks list. 
# Then makes the previously initialized task variable global and assigns value of the given Task to it 
def chooseTask(Task):
    global task
    task=onto.search(iri=str(Task))[0]
    
# Takes an Object of the object list.
# Then makes the previously initialized tobject variable global and assigns value of the given Object to it 
def chooseObject(Object):
    global tobject
    tobject=getattr(OBO,str(Object).split(":")[1])
    
# Create the dropdown widgets
task_widget = widgets.Dropdown(options=tasks, description='Task:')
object_widget = widgets.Dropdown(options=objects, description='Object:')

# Define the event handlers
def taskEvent(event):
    chooseTask(event.new)

def objectEvent(event):
    chooseObject(event.new)
    
# Attach the event handlers to the widgets
task_widget.observe(taskEvent, names='value')
object_widget.observe(objectEvent, names='value')

# Combine the widgets using HBox
widgets_display = HBox([task_widget, object_widget])

# Display widgets
widgets_display

In [ ]:
print(task)
print(tobject)

# Food-Object-centered Queries
- The following functions are centered on a food object and therefore a food object class is required.

- Utility function to retrieve all restrictions, the restrictions are structured as parent classes of the food object.
- requires food object class as input, for example: OBO.FOODON_03301710

In [ ]:
def get_restrictions_of_class(cls):
    union = set()
    # Looks for the superclasses of the input class.
    for parent in cls.is_a:
        # Checks if it is a restriction
        if not isinstance(parent, owlready2.Restriction):
            # If the input class has inherited superclasses we have to look for its 
            # parent class and retrieve the restrictions from there.
            for e in parent.is_a:
                if isinstance(e, owlready2.Restriction):
                    union.add(e)
        else:
            union.add(parent)
    
    return list(union)

print(get_restrictions_of_class(tobject))

- Query for the required Tool, requires a food object as input.
- The food object class has a restriction which shows which CuttingTool is required.
- First we consider only the restrictions with the property: "SOMA.hasDisposition" and check if the food object can be cut.
- Secondly we check if a CuttingTask can be afforded, by looking at the property "SOMA.affordsTask".
- Finally we can retrieve the required tool with the "SOMA.affordsTrigger" property.

In [ ]:
def gettool_query(food_object):
    # Calls the restriction of the input class
    restrictions = get_restrictions_of_class(food_object)
    # Iterate over the restrictions
    for restriction in restrictions:
        # Check for the SOMA.hasDisposition restriction
        if isinstance(restriction, owlready2.Restriction) and restriction.property == SOMA.hasDisposition:
            for r in restriction.value.Classes:
                # Check if the Cutability restriction is defined for the input class.
                if r == CUT2.Cuttability:
                    for c in restriction.value.Classes:
                        # Check for the SOMA.hasDisposition restriction
                        if isinstance(c, owlready2.Restriction) and c.property == SOMA.affordsTask:
                            # If there is no task found return "No CuttingTask found"
                            if c.value != CUT.CuttingTask:
                                return "No CuttingTask found"
                        # Check for the SOMA.affordsTrigger restriction and return its value.
                        if isinstance(c, owlready2.Restriction) and c.property == SOMA.affordsTrigger:
                            return c.value.value

print(gettool_query(tobject))

- This function checks if and which prior actions are required, for example if the core of a fruit needs to be removed.
- First we want to check if an action is required by looking at the "CUT.hasPart" property, this property contains classes like "Cut.ShouldBeAvoided" or "MustBeAvoided", which indicates that a prior action is required.
- If the case occurs, we look at the Class which is retrieved from the "CUT.hasPart" property, like "Stem" and "Core". This classes also contain restrictions from which we can retrieve the needed action, by asking the property "SOMA.affordsTask".
- The output can be for example "StemRemoving" or "CoreCutting"

In [ ]:
def needed_action_query(food_object):
    needed_actions = []
    # Calls the restriction of the input class
    restrictions = get_restrictions_of_class(food_object)
    # Iterate over the restrictions
    for i in restrictions:
        # Check if the input class contains the "hasPart" restriction
        if isinstance(i, owlready2.Restriction) and i.property == CUT.hasPart:
            # "hasPart" Restriction contains nested restriction itself,
            # so it is required to iterate over them as well
            for c in i.value.Classes:
                # Looking for the "hasEdibility" restriction
                if isinstance(c, owlready2.Restriction) and c.property == CUT.hasEdibility:
                    # If one of the 2 cases occur, it is required to iterate over the parent restriction to 
                    # retrieve the needed action. This happens by getting the "hasPart" property, like "Stem" or "Core".
                    if c.value == CUT.ShouldBeAvoided or c.value == CUT.MustBeAvoided:
                        for j in i.value.Classes:
                            # We look for classes that are not a restriction itself
                            if not isinstance(j, owlready2.Restriction):
                                # Input class inherits from its parent class, and therefore 
                                # we have to check the restrictions of the parent class.
                                if len(get_restrictions_of_class(j)) == 0:
                                    continue
                                if len(get_restrictions_of_class(j)) > 0:
                                    # Iterate over the restrictions of the retrieved class.
                                    for h in get_restrictions_of_class(j):
                                        for k in h.value.Classes:
                                            # Check for the affordsTask restriction that indicates which action is required.
                                            if not isinstance(k, owlready2.Restriction):
                                                continue
                                            if isinstance(k, owlready2.Restriction) and k.property == SOMA.affordsTask:
                                                needed_actions.append(k.value)


    return needed_actions

print(needed_action_query(tobject))

- Returns the required tool for the needed action provided by the function: "need_action_query"

In [ ]:
def needed_action_tool_query(food_object):
    result = {}
    # Returns the restrictions of the food object
    restrictions = get_restrictions_of_class(food_object)
    # Queries the action(s) required for the food object
    actions = needed_action_query(food_object)
    
    # Iterates over the restriction of the food object
    for restriction in restrictions:
        # Check for the hasDisposition Restriction, which contains the affordsTask Restriction.
        if isinstance(restriction, owlready2.Restriction) and restriction.property == SOMA.hasDisposition:
            for r in restriction.value.Classes:
                if isinstance(r, owlready2.Restriction) and r.property == SOMA.affordsTask:
                    # Checks if the restriction value is included in the required actions.
                    if r.value in actions:
                        needed_action = r.value
                        for i in restriction.value.Classes:
                            if isinstance(i, owlready2.Restriction) and i.property == SOMA.affordsTrigger:
                                # Write an element into the dictionary, where the key corresponds to the required action, 
                                # and the value correspond to the required tool for the action.
                                result[needed_action] = i.value.value
    
    return result
                            
                            
print(needed_action_tool_query(tobject))

# Task-centered Queries
- The functions are recursive
- In some cases the Cutting Task Class does not have any restrictions but inherit them from their parent class
- This must be checked first, if the given class has no restrictions, the parent class is given as argument and the function is called again.

- Returns the prior task, if required, of a given task, by accessing the "CUT.requiresPriorTask" property.

In [ ]:
def get_prior_task(task):
    restrictions = [i for i in task.is_a if isinstance(i, owlready2.Restriction)]
    # If there is no restrictions over the given input class, 
    # call the function recursively and check for its parent class.
    if len(restrictions) == 0:
        if task.is_a:
            return get_prior_task(task.is_a[0])
    
    # Return the value of the "requiresPriorTask" restriction if available.
    if len(restrictions) < 0:
        for i in task.is_a:
            if isinstance(i, owlready2.Restriction) and i.property == CUT.requiresPriorTask:
                return i.value
            
print(get_prior_task(task))

- Returns the number of repetitions required for a given task, by accessing the "CUT.repetitions" property.

In [ ]:
def get_number_of_repetitions(task):
    restrictions = [i for i in task.is_a if isinstance(i, owlready2.Restriction)]
    # If there is no restrictions over the given input class, 
    # call the function recursively and check for its parent class.
    if len(restrictions) == 0:
        if task.is_a:
            return get_number_of_repetitions(task.is_a[0])
    
    # Return the value of the "repetitions" restriction if available.
    if len(restrictions) > 0:
        for i in task.is_a:
            if isinstance(i, owlready2.Restriction) and i.property == CUT.repetitions:
                if i.cardinality:
                    return "min repetitions:" + str(i.cardinality)
                else:
                    return i.value




print(get_number_of_repetitions(task))

- Returns the position of execution, by accessing the "CUT.affordsPosition" property.

In [ ]:
def get_position_of_execution(task=None):
    restrictions = [i for i in task.is_a if isinstance(i, owlready2.Restriction)]
    # If there is no restrictions over the given input class, 
    # call the function recursively and check for its parent class.
    if len(restrictions) == 0:
        if task.is_a:
            return get_position_of_execution(task.is_a[0])
    # Return the value of the "affordsPosition" restriction if available.
    if len(restrictions) > 0:
        for i in restrictions:
            if i.property == CUT.affordsPosition:
                return i.value

print(get_position_of_execution(task))

## Test

- Testing the Queries for every task and every FoodObject

In [ ]:
tasks = [('Cutting Action',"cut:CuttingAction"),
        ('Quartering', "cut:Quartering"),
        ('Julienning',"cut:Julienning"),
        ('Halving',"cut:Halving"),
        ('Dicing',"soma:Dicing"),
        ('Cutting',"soma:Cutting"),
        ('Slicing',"soma:Slicing"),
        ('Snipping',"cut:Snipping"),
        ('Slivering',"cut:Slivering"),
        ('Sawing',"cut:Sawing"),
        ('Paring',"cut:Paring"),
        ('Carving',"cut:Carving"),
        ('Mincing',"cut:Mincing"),
        ('Cubing',"cut:Cubing"),
        ('Chopping',"cut:Chopping")]

objects=[('almond', "obo:FOODON_00003523"),
        ('apple', "obo:FOODON_03301710"),
        ('avocado', "obo:FOODON_00003600"),
        ('banana', "obo:FOODON_00004183"),
        ('bean', "obo:FOODON_03301403"),      
        ('citron', "obo:FOODON_03306596"),
        ('coconut', "obo:FOODON_00003449"),     
        ('cucumber', "obo:FOODON_00003415"),
        ('kiwi', "obo:FOODON_00004387"), 
        ('kumquat', "obo:FOODON_03306597"),
        ('lemon', "obo:FOODON_03301441"),
        ('lime', "obo:FOODON_00003661"),
        ('olive', "obo:FOODON_03317509"),
        ('orange', "obo:FOODON_03309832"),
        ('peach', "obo:FOODON_03315502"), 
        ('pepper', "obo:FOODON_00003520"),
        ('pineapple', "obo:FOODON_00003459"),
        ('pumpkin', "obo:FOODON_00003486"),
        ('strawberry', "obo:FOODON_00003443"),        
        ('squash', "obo:FOODON_00003539"),
        ('tomato', "obo:FOODON_03309927")]

# Task Queries Tests
- Replace the task with the wished task to test.

In [ ]:
task = SOMA.Slicing

print(get_number_of_repetitions(task))
print(get_position_of_execution(task))
print(get_prior_task(task))

# Food Object Tests
- Iterate over every defined object and apply the implemented query-functions.

In [ ]:
for o in objects:
    foodOn_cls = getattr(OBO,o[1].split(":")[1])
    print(foodOn_cls)
    print(foodOn_cls.label)
    print(gettool_query(foodOn_cls))
    print(needed_action_query(foodOn_cls))
    print("New Cls")
